<a href="https://colab.research.google.com/github/mgersins-design/Machine-Learning/blob/main/HW7/ConvNN_CIFAR_10_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

import datetime

In [20]:
transform_process = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    )
])

data_path = './data'

cifar10_train = datasets.CIFAR10(
    data_path, train=True, download=True, transform=transform_process
)
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True, transform=transform_process
)

batch_size = 256
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers = 2)
val_loader = DataLoader(cifar10_val, batch_size=batch_size, shuffle=False, num_workers = 2)

print(f"Trainingsdaten: {len(cifar10_train)} Bilder")
print(f"Validierungsdaten: {len(cifar10_val)} Bilder")

Trainingsdaten: 50000 Bilder
Validierungsdaten: 10000 Bilder


In [21]:
# Creating the model
class Net (nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, padding = 1)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16, 8, kernel_size = 3, padding = 1)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8, 32)
    self.act3 = nn.ReLU()
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = out.view(-1, 8*8*8)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [22]:
# Switching to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [23]:
def train_and_validate(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
  device = next(model.parameters()).device

  for epoch in range(1, n_epochs+1):
    # Training Phase
    model.train()
    loss_train = 0.0
    correct_train = 0
    total_train = 0

    for imgs, labels in train_loader:
      imgs = imgs.to(device = device)
      labels = labels.to(device = device)

      outputs = model(imgs)
      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
      #_, predicted = torch.max(outputs, dim=1)
      #total_train += labels.size(0)
      #correct_train += int((predicted == labels).sum())

    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
      for imgs, labels in val_loader:
        imgs = imgs.to(device = device)
        labels = labels.to(device = device)

        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total_val += labels.shape[0]
        correct_val += int((predicted == labels).sum())

    avg_loss = loss_train / len(train_loader)
    #acc_train = correct_train / total_train
    acc_val = correct_val / total_val

    if epoch == 1 or epoch % 10 == 0:
      print('{} Epoch {:3d}, Training loss {:.4f}, Val Acc: {:.2f}'.format(
          datetime.datetime.now(), epoch, avg_loss, acc_val))


In [24]:
model = Net().to(device = device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()

train_and_validate(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    val_loader= val_loader
)

2025-11-25 01:32:46.840947 Epoch   1, Training loss 1.7827, Val Acc: 0.43
2025-11-25 01:33:51.547026 Epoch  10, Training loss 1.0788, Val Acc: 0.62
2025-11-25 01:35:02.664372 Epoch  20, Training loss 0.9604, Val Acc: 0.64
2025-11-25 01:36:13.738978 Epoch  30, Training loss 0.8899, Val Acc: 0.66
2025-11-25 01:37:25.064815 Epoch  40, Training loss 0.8488, Val Acc: 0.65
2025-11-25 01:38:36.651168 Epoch  50, Training loss 0.8205, Val Acc: 0.66
2025-11-25 01:39:48.647218 Epoch  60, Training loss 0.7913, Val Acc: 0.66
2025-11-25 01:41:00.124623 Epoch  70, Training loss 0.7814, Val Acc: 0.65
2025-11-25 01:42:12.124882 Epoch  80, Training loss 0.7618, Val Acc: 0.66
2025-11-25 01:43:23.864381 Epoch  90, Training loss 0.7478, Val Acc: 0.66
2025-11-25 01:44:34.872060 Epoch 100, Training loss 0.7330, Val Acc: 0.66


In [25]:
# Model Complexity
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18354, [432, 16, 1152, 8, 16384, 32, 320, 10])